## Simulate DaskClassifier based on Timing Data

In [1]:
import os
import torch
import torchvision
import torchvision.transforms as transforms
import csv
import importlib
import pandas as pd
from time import time
from copy import copy
from model import Wide_ResNet

In [2]:
# import simulator
os.chdir('..')
classifier = importlib.import_module("exp-dask.classifier")
from classifier import DaskClassifierSimulator

In [3]:
# import
from dask.distributed import Client
# training client
def _prep():
    from distributed.protocol import torch

client = Client(processes=False, n_workers=16, threads_per_worker=1, memory_limit="32GB")
client.run(_prep)

#from model import Wide_ResNet
client.upload_file("./exp-dask/model.py")

client

Client Scheduler: inproc://172.31.16.127/4804/1 Dashboard: http://172.31.16.127:8787/status,Cluster Workers: 16 Cores: 16 Memory: 512.00 GB


In [4]:
# Gather and prepare data
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])
train_set = torchvision.datasets.CIFAR10(root='./exp-dask/data', train=True, download=True, transform=transform_train)
test_set = torchvision.datasets.CIFAR10(root='./exp-dask/data', train=False, download=True, transform=transform_test)
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [5]:
def write_stats(hist, directory, filename):
    with open('./exp-dask/{}/{}'.format(directory, filename), 'w', encoding='utf8', newline='') as output_file:
        fc = csv.DictWriter(output_file, fieldnames=hist[0].keys())
        fc.writeheader()
        fc.writerows(hist)

In [6]:
def train(model, train_set, test_set, stats_df, n_epochs=200, log_interval=1, stats_dir=''):
    """
    Train based on expiriment params
    """
    
    # stats to load into the simulator
    stats = list(stats_df.T.to_dict().values())
    print("[SETUP] Loaded {} epochs of stats".format(len(stats)))
    
    # preprocess data
    start = time()
    train_set = model.preprocess(train_set)
    test_set = model.preprocess(test_set)
    print("[SETUP] Pre-Processed in {} seconds".format(time() - start))
    
    history = []
    # run simulations over all sim data
    for epoch in range(min(n_epochs, len(stats))):
        
        start = time()
        
        # set stats
        stat = stats.pop(0)
        model.set_sim(stat)
        
        # run
        print("[Epoch {}]".format(epoch), end="")
        model.partial_fit(train_set)
        score = model.score(test_set)
        
        # store data
        datum = {"epoch": epoch, "score": score, **model.get_params(), **model.meta_}
        print(" Score: {} in {} seconds".format(score, time() - start))
        history.append(datum)
        
        if epoch % log_interval == 0:
            # write_stats(history, stats_dir, 'results-ep{}.csv'.format(epoch))
            pass

    return history

In [7]:
def run_sim(stats_df_loc, out_folder, timings, n_epochs=200, grads_per_worker=128, max_bs=99999999):
    """
    Sets up the simulator model, transforms stats (to account for changes in the base classifier and how
    it saves data), and calls the train function.
    """
    
    print("Running simulation....\n\n")
    start = time()
    
    # stats from an actual training run
    stats_df = pd.read_csv(stats_df_loc)
    
    # old versions of the DaskClassifier had different collumn names
    if 'lr_' in stats_df.columns:
        stats_df = stats_df.rename(columns={"lr_": "partial_fit__lr", "batch_size_": "partial_fit__batch_size"})
    
    # pull initial stats from the CSV
    init_lr = stats_df['partial_fit__lr'][0]
    init_bs = stats_df['partial_fit__batch_size'][0]
    momentum = stats_df['optimizer__momentum'][0]
    device = "cpu" if not torch.cuda.is_available() else "cuda:0"

    # args
    args = dict(
        module=Wide_ResNet,
        module__depth=16,
        module__widen_factor=4,
        module__dropout_rate=0.3,
        module__num_classes=len(classes),
        loss=torch.nn.CrossEntropyLoss,
        optimizer=torch.optim.SGD,
        optimizer__lr=init_lr,
        optimizer__momentum=momentum,
        optimizer__nesterov=True,
        optimizer__weight_decay=0.5e-3,
        batch_size=init_bs,
        max_epochs=200,
        device=device,
        grads_per_worker=grads_per_worker,
        client=client,
        lr=init_lr,
        max_batch_size=max_bs
    )
    
    # create model and set initial timings
    model = DaskClassifierSimulator(**args)
    model.set_times(timings['mult'], timings['score'], timings['deepcopy'], timings['grad128'])
    
    # train based on normal train function
    hist = train(model, 
             train_set, 
             test_set, 
             stats_df,
             n_epochs=n_epochs, 
             log_interval=20,
             stats_dir=out_folder
            )
    # write
    with open(out_folder, 'w', encoding='utf8', newline='') as output_file:
        fc = csv.DictWriter(output_file, fieldnames=hist[0].keys())
        fc.writeheader()
        fc.writerows(hist)
    
    print('Finished simulation in {} seconds'.format(time() - start))
    

In [ ]:
# test exp
timings = { 'mult': True, 'score': 0.0, 'deepcopy': 0.0, 'grad128': 0.0 }
stats_path = './exp-dask/stats/increasing-bs-final.csv'
out_path = 'sim-results/test'
run_sim(stats_path, out_path, timings, n_epochs=1)

# Siming grads per worker
Simulate the bs=512 experiments with differen numbers of grads per worker

In [8]:
## Timings for new simulation
SCORE_TIME = 0.0 # 5.39407711148262
DEEPCOPY_TIME = 0.127488e-3  # seconds
GRAD_TIME_128 = 0.5 * 78.32e-3
timings = { 'mult': True, 'score': SCORE_TIME, 'deepcopy': DEEPCOPY_TIME, 'grad128': GRAD_TIME_128 }

In [9]:
stats_path = './exp-dask/stats/dec-lr-512bs-final.csv'
for grads_per_worker in [32, 64, 128, 256, 512]:
    out_path = 'exp-dask/sim-results/grads_per_worker/grads_per_worker_{}-final.csv'.format(grads_per_worker)
    run_sim(stats_path, out_path, timings, n_epochs=200, grads_per_worker=grads_per_worker)

Running simulation....


[SETUP] Loaded 200 epochs of stats
[SETUP] Pre-Processed in 19.061187744140625 seconds
[Epoch 0] Score: 0.489799976348877 in 13.980453729629517 seconds
[Epoch 1] Score: 0.6232999563217163 in 10.961429119110107 seconds
[Epoch 2] Score: 0.6603999733924866 in 10.977453470230103 seconds
[Epoch 3] Score: 0.7120999693870544 in 11.118677377700806 seconds
[Epoch 4] Score: 0.7465999722480774 in 11.09188437461853 seconds
[Epoch 5] Score: 0.7512999773025513 in 11.020839929580688 seconds
[Epoch 6] Score: 0.7569999694824219 in 10.94118857383728 seconds
[Epoch 7] Score: 0.7965999841690063 in 11.161171913146973 seconds
[Epoch 8] Score: 0.7978999614715576 in 11.282478094100952 seconds
[Epoch 9] Score: 0.8180999755859375 in 10.946798086166382 seconds
[Epoch 10] Score: 0.8219999670982361 in 10.951257944107056 seconds
[Epoch 11] Score: 0.8235999941825867 in 11.395576477050781 seconds
[Epoch 12] Score: 0.8337999582290649 in 11.041080236434937 seconds
[Epoch 13] Score: 0.8313999772

distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)


 Score: 0.905299961566925 in 12.330555200576782 seconds
[Epoch 95] Score: 0.9052000045776368 in 10.896064519882202 seconds
[Epoch 96]

distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)


 Score: 0.9052000045776368 in 12.31101393699646 seconds
[Epoch 97] Score: 0.9028999805450441 in 10.958357572555542 seconds
[Epoch 98] Score: 0.9045999646186828 in 10.958157300949097 seconds
[Epoch 99]

...

 Score: 0.8851000070571899 in 10.249308109283447 seconds
[Epoch 56] Score: 0.8819999694824219 in 7.6733479499816895 seconds
[Epoch 57] Score: 0.882099986076355 in 7.661723375320435 seconds
[Epoch 58] Score: 0.8784999847412109 in 7.652313232421875 seconds
[Epoch 59] Score: 0.8831999897956848 in 7.660414934158325 seconds
[Epoch 60] Score: 0.9025999903678894 in 7.6720263957977295 seconds
[Epoch 61] Score: 0.9042999744415284 in 7.65195369720459 seconds
[Epoch 62] Score: 0.9083999991416932 in 7.640193462371826 seconds
[Epoch 63] Score: 0.9065999984741212 in 7.699592590332031 seconds
[Epoch 64] Score: 0.9041999578475952 in 7.669189453125 seconds
[Epoch 65] Score: 0.9070999622344972 in 7.629862308502197 seconds
[Epoch 66] Score: 0.9077999591827391 in 7.670933246612549 seconds
[Epoch 67] Score: 0.9038999676704408 in 7.683232307434082 seconds
[Epoch 68] Score: 0.9078999757766724 in 7.678803205490112 seconds
[Epoch 69]

distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)


 Score: 0.90829998254776 in 10.335681915283203 seconds
[Epoch 70] Score: 0.9052000045776368 in 7.691906690597534 seconds
[Epoch 71] Score: 0.9041000008583068 in 7.692845582962036 seconds
[Epoch 72] Score: 0.9052000045776368 in 7.628974676132202 seconds
[Epoch 73] Score: 0.9065999984741212 in 7.64960503578186 seconds
[Epoch 74] Score: 0.9071999788284302 in 7.670116662979126 seconds
[Epoch 75] Score: 0.903599977493286 in 7.673335790634155 seconds
[Epoch 76] Score: 0.9041000008583068 in 7.698579788208008 seconds
[Epoch 77] Score: 0.9049999713897704 in 7.676590919494629 seconds
[Epoch 78] Score: 0.9048999547958374 in 7.669848918914795 seconds
[Epoch 79] Score: 0.9080999493598938 in 7.6497344970703125 seconds
[Epoch 80] Score: 0.9048999547958374 in 7.6478564739227295 seconds
[Epoch 81] Score: 0.9057999849319458 in 7.631267309188843 seconds
[Epoch 82] Score: 0.9047999978065492 in 7.629146337509155 seconds
[Epoch 83]

distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)


 Score: 0.901799976825714 in 10.408070087432861 seconds
[Epoch 84] Score: 0.9053999781608582 in 7.69789457321167 seconds
[Epoch 85] Score: 0.9061999917030334 in 7.652296304702759 seconds
[Epoch 86] Score: 0.9088000059127808 in 7.645434379577637 seconds
[Epoch 87] Score: 0.9049999713897704 in 7.636725664138794 seconds
[Epoch 88] Score: 0.9025999903678894 in 7.676135301589966 seconds
[Epoch 89] Score: 0.9055999517440796 in 7.650813817977905 seconds
[Epoch 90] Score: 0.9070000052452089 in 7.64389705657959 seconds
[Epoch 91] Score: 0.9059999585151672 in 7.663691520690918 seconds
[Epoch 92] Score: 0.9067999720573424 in 7.676034688949585 seconds
[Epoch 93] Score: 0.9062999486923218 in 7.624976396560669 seconds
[Epoch 94] Score: 0.905299961566925 in 7.6332173347473145 seconds
[Epoch 95] Score: 0.9052000045776368 in 7.6653220653533936 seconds
[Epoch 96] Score: 0.9052000045776368 in 7.6248719692230225 seconds
[Epoch 97] Score: 0.9028999805450441 in 7.684947490692139 seconds
[Epoch 98] Score: 0.

## Simulations with faster network bandwidth #2 (comment).

In [11]:
## Timings for new simulation
SCORE_TIME = 0.0 # 5.39407711148262
DEEPCOPY_TIME = 0.127488e-3  # seconds
GRAD_TIME_128 = 78.32e-3  # seconds
timings = { 'mult': True, 'score': SCORE_TIME, 'deepcopy': DEEPCOPY_TIME, 'grad128': GRAD_TIME_128 }

In [ ]:
# exp 2.1 - dec-lr-P-machine-fast
stats_path = './exp-dask/stats/decreasing-lr/exp--final.csv'
out_path = 'sim-results/dec-lr-mult-machines-fast'
run_sim(stats_path, out_path, timings)

In [ ]:
# exp 2.2 - increasing batch p machines
stats_path = './exp-dask/stats/increasing-bs/exp-final.csv'
out_path = 'sim-results/inc-bs-mult-machines-fast'
run_sim(stats_path, out_path, timings)

In [ ]:
# exp 2.3 - hybrid 1 P machine
stats_path = './exp-dask/stats/hybrid/exp-final.csv'
out_path = 'sim-results/hybrid1-mult-machines-fast'
run_sim(stats_path, out_path, timings)

In [ ]:
# exp 2.4 - hybrid 2 P machine
stats_path = './exp-dask/stats/hybrid-2/exp-final.csv'
out_path = 'sim-results/hybrid2-mult-machines-fast'
run_sim(stats_path, out_path, timings)

In [ ]:
# exp 11 - large bs 1 P machine
stats_path = './exp-dask/stats/large-bs-0/exp-final.csv'
out_path = 'sim-results/large-bs1-mult-machines-fast'
run_sim(stats_path, out_path, timings, max_bs=5120)

In [ ]:
# exp 12 - large bs 2 P machine
stats_path = './exp-dask/stats/large-bs-1/exp-final.csv'
out_path = 'sim-results/large-bs2-mult-machines-fast'
run_sim(stats_path, out_path, timings, max_bs=5120)







## Simulations for 1 machine with normal network bandwidth

In [11]:
## TIMINGS FOR NORMAL EXPERIMENTS
SCORE_TIME = 0.0 # 5.39407711148262
DEEPCOPY_TIME = 0.05855  # seconds
GRAD_TIME_128 = 0.07832  # seconds
timings = { 'mult': False, 'score': SCORE_TIME, 'deepcopy': DEEPCOPY_TIME, 'grad128': GRAD_TIME_128 }

In [ ]:
# exp 1 - dec-lr-1-machine 
stats_path = './exp-dask/stats/decreasing-lr/exp--final.csv'
out_path = 'sim-results/dec-lr-1-machine'
run_sim(stats_path, out_path, timings)

In [ ]:
# exp 5 - large bs 1 1 machine
stats_path = './exp-dask/stats/large-bs-0/exp-final.csv'
out_path = 'sim-results/large-bs1-1-machine'
run_sim(stats_path, out_path, timings, max_bs=5120)

In [ ]:
# exp 2 - inc-bs
stats_path = './exp-dask/stats/increasing-bs/exp-final.csv'
out_path = 'sim-results/inc-bs-1-machine'
run_sim(stats_path, out_path, timings)

In [ ]:
# exp 3 - hybrid 1 1 machine
stats_path = './exp-dask/stats/hybrid/exp-final.csv'
out_path = 'sim-results/hybrid1-1-machine'
run_sim(stats_path, out_path, timings)

In [ ]:
# exp 4 - hybrid 2 1 machine
stats_path = './exp-dask/stats/hybrid-2/exp-final.csv'
out_path = 'sim-results/hybrid2-1-machine'
run_sim(stats_path, out_path, timings)

In [ ]:
# exp 6 - large bs 2 1 machine
stats_path = './exp-dask/stats/large-bs-1/exp-final.csv'
out_path = 'sim-results/large-bs2-1-machine'
run_sim(stats_path, out_path, timings, max_bs=5120)

## Simulations for multiple machines with normal network bandwidth

In [16]:
# timings for multi machine set ups
SCORE_TIME = 0.0 # 5.39407711148262
DEEPCOPY_TIME = 0.0 # set internall based on N workers
GRAD_TIME_128 = 0.07832  # seconds
timings = { 'mult': True, 'score': SCORE_TIME, 'deepcopy': DEEPCOPY_TIME, 'grad128': GRAD_TIME_128 }

In [ ]:
# exp 11 - large bs 1 P machine
stats_path = './exp-dask/stats/large-bs-0/exp-final.csv'
out_path = 'sim-results/large-bs1-mult-machines'
run_sim(stats_path, out_path, timings, max_bs=5120)

In [ ]:
# exp 7 - dec-lr-P-machine 
stats_path = './exp-dask/stats/decreasing-lr/exp--final.csv'
out_path = 'sim-results/dec-lr-mult-machines'
run_sim(stats_path, out_path, timings)

In [ ]:
# exp 8 - increasing batch p machines
stats_path = './exp-dask/stats/increasing-bs/exp-final.csv'
out_path = 'sim-results/inc-bs-mult-machines'
run_sim(stats_path, out_path, timings)

In [ ]:
# exp 9 - hybrid 1 P machine
stats_path = './exp-dask/stats/hybrid/exp-final.csv'
out_path = 'sim-results/hybrid1-mult-machines'
run_sim(stats_path, out_path, timings)

In [ ]:
# exp 10 - hybrid 2 P machine
stats_path = './exp-dask/stats/hybrid-2/exp-final.csv'
out_path = 'sim-results/hybrid2-mult-machines'
run_sim(stats_path, out_path, timings)

In [ ]:
# exp 12 - large bs 2 P machine
stats_path = './exp-dask/stats/large-bs-1/exp-final.csv'
out_path = 'sim-results/large-bs2-mult-machines'
run_sim(stats_path, out_path, timings, max_bs=5120)